This is a Logistic version. We use categorical variable. We use OneHotEncoder to deal with the 

In [4]:
# -*- coding: utf-8 -*-
import zipfile
import pandas as pd
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

# load data
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
dfAd = pd.read_csv("ad.csv")

# process data
dfTrain = pd.merge(dfTrain, dfAd, on="creativeID")
dfTest = pd.merge(dfTest, dfAd, on="creativeID")
y_train = dfTrain["label"].values

# feature engineering/encoding
enc = OneHotEncoder()
feats = ["creativeID", "adID", "camgaignID", "advertiserID", "appID", "appPlatform"]
for i,feat in enumerate(feats):
    x_train = enc.fit_transform(dfTrain[feat].values.reshape(-1, 1))
    x_test = enc.transform(dfTest[feat].values.reshape(-1, 1))
    if i == 0:
        X_train, X_test = x_train, x_test
    else:
        X_train, X_test = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test))

In [12]:
print(X_train.tocsr()[5])
print("The shape of X_train is:",X_train.shape)

  (0, 2966)	1.0
  (0, 7575)	1.0
  (0, 9841)	1.0
  (0, 10447)	1.0
  (0, 10574)	1.0
  (0, 10578)	1.0
The shape of X_train is: (3749528, 10580)


We can see that train feature is a large sparse matrix. To store the matrix, we use index to indentify which numbers should be 1. In this way, the computer can save lots of space.

In [13]:
# model training
lr = LogisticRegression()
lr.fit(X_train, y_train)
proba_test = lr.predict_proba(X_test)[:,1] # the first column is probability for 0 and second column is for 1

# submission
df = pd.DataFrame({"instanceID": dfTest["instanceID"].values, "proba": proba_test})
df.sort_values("instanceID", inplace=True)
df.head()

,instanceID,proba
0,1,0.001536
1238,2,0.002878
1887,3,0.025357
12634,4,0.025874
35772,5,0.001786


In [ ]:
# you can also save your result
df.to_csv("submission.csv", index=False)
with zipfile.ZipFile("submission.zip", "w") as fout:
    fout.write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)